In [68]:
from pprint import pprint
import pickle
import re
from utils import *
import time#计时模块
from tqdm import tqdm
from neo4j import GraphDatabase
# 连接数据库
driver = GraphDatabase.driver("bolt://localhost:7687")
print('connect ok!')

connect ok!


In [2]:
f = open('corpus/task6ckbqa_dev_2019.questions','r',encoding='utf-8').readlines()
men2ent_dic = get_mention2entity_list()
predata = {}
for i in range(len(f)):
    predata[i]={}
    predata[i]['question']=f[i].split(':',1)[1].rstrip('\n')

In [69]:
def template_no1():
    #解决“出了哪些”的问题
    template = "(.*学)出[过|了]哪些(.*)？"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group(1),match_obj.group(2))
            e1 = match_obj.group(1)
            m2 = match_obj.group(2)
            for one_mention in men2ent_dic[m2]:
                if one_mention[1] == '1' or '0':
                    e2 = one_mention[0]
                    break
            print(i,e1,e2)
            #查询库
            session = driver.session()
            cql = 'match (a:Entity)<-[r1:Relation]-(b:Entity)-[r2:Relation]-(c:Entity) where a.name=$name and r1.name = $name2 and c.name=$name3 return b.name'
            result = session.run(cql,name='<'+e1+'>',name2='<毕业院校>',name3='<'+e2+'>')
            for x in result:
                print(x['b.name'])
                result_dict[i].append(x['b.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [70]:
def template_no2():
    #解决“出了哪些”的问题
#     template = "(.*有多少家.*)|(.*有哪几家.*)" 10 "(.*什么学历.*)" 7 "(.*期货交易所.*)" 2类
    template ="([0-9]+)的(.*)是.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group(1),match_obj.group(2))
            e1 = match_obj.group(1)
            m2 = match_obj.group(2)
            #查询库
            session = driver.session()
            cql = 'match (a:Entity)-[r1:Relation]-(b:Entity)-[r2:Relation]-(c:Entity) where a.name=$name and r1.name = $name2 and r2.name=~".*'+m2[:2]+'.*" return c.name'
            result = session.run(cql,name='<'+e1+'>',name2='<股票代码>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [71]:
def template_no3():
    #解决“出了哪些”的问题
    template = "([^的]*).*董事长.*什么学历.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group(1))
            e1 = match_obj.group(1)
            #查询库
            session = driver.session()
            cql = 'match (a:Entity)-[r1:Relation]-(b:Entity)-[r2:Relation]-(c:Entity) where a.name=$name and r1.name = $name2 and r2.name=$name3 return c.name'
            result = session.run(cql,name='<'+e1+'>',name2='<董事长>',name3='<学历>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [72]:
def template_no4():
    #解决“出了哪些”的问题
    template = "位于(..)的期货交易所.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group(1))
            e1 = match_obj.group(1)
            ##############
            if e1=='上海':
                e1='魔都'
            ##############
            #查询库
            session = driver.session()
            cql = 'match (a:Entity)-[r1:Relation]-(b:Entity) where a.name=$name and r1.name = $name2 return b.name'
            result = session.run(cql,name='<'+e1+'>',name2='<期货交易所>')
            for x in result:
                print(x['b.name'])
                result_dict[i].append(x['b.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [73]:
def template_no5():
    #解决“出了哪些”的问题
    template = "(.*)有.*期货交易所.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group(1))
            e1 = match_obj.group(1)
            ##############
            if e1=='国内'or e1=='中国':
                session = driver.session()
                cql = 'match (a:Entity)<-[r1:Relation]-(b:Entity) where a.name="<期货交易所>" and r1.name = "<类别>" return b.name'
                result = session.run(cql)
                for x in result:
                    print(x['b.name'])
                    result_dict[i].append(x['b.name'])
                count+=1
            else:   
                ##############
                #查询库
                session = driver.session()
                cql = "match (a:Entity)-[b:Relation]-(c:Entity)-[d:Relation]-(e:Entity)\
                            where a.name='<期货交易所>' and b.name='<类别>'and d.name='<位置>'and e.name=$name return c.name"
                                
                result = session.run(cql,name='<'+e1+'>')
                for x in result:
                    print(x['c.name'])
                    result_dict[i].append(x['c.name'])

                count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [74]:
def template_no6():
    #解决“出了哪些”的问题
    template = "(.*)上市的(期货品种中)?(.*)有什么"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group())
            e1 = match_obj.group(1)
            e2 = match_obj.group(3)

            #查询库
            session = driver.session()
            cql = "match (a:Entity)-[b:Relation]-(c:Entity)-[d:Relation]-(e:Entity)\
                        where a.name=$name and d.name='<类别>'and e.name=$name1 return c.name"

            result = session.run(cql,name='<'+e1+'>',name1='<'+e2+'>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict
template_no6()

679 郑商所上市的农产品期货有什么
<棉纱>
<苹果>
<棉花>
<菜籽油>
<油菜籽>
<强麦>
<普麦>
<晚籼稻>
<白糖>
<粳稻>
<早籼稻>
<菜籽粕>
680 郑商所上市的非农产品期货有什么
<硅铁>
<动力煤>
<甲醇>
<玻璃>
<PTA>
<锰硅>
681 上海期货交易所上市的期货品种中金属期货有什么
<镍>
<镍>
<铜>
<铜>
<黄金>
<锌>
<锌>
<锡>
<锡>
<铝>
<铝>
<螺纹钢>
<螺纹钢>
<热轧卷板>
<铅>
<铅>
<线材>
<线材>
<白银>
682 上海期货交易所上市的期货品种中能源化工期货有什么
<天然橡胶>
<燃料油>
<沥青>
<沥青>
<纸浆>
<原油>
683 上海期货交易所上市的期货品种中衍生品期货有什么
<铜期权>
<天胶期权>
此类问题所占比例为：0.6527415143603134 5/766


{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [],
 65: [],
 66: [],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [],
 79: [],
 80: [],
 81: [],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: [],
 88: [],
 89: [],
 90: [],
 91: [],
 92: [],
 93: [],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [],
 100: [],
 101: [],
 102: [],
 103: [],
 104: [],
 105: [],
 106: [],
 107: [],
 108: [],
 109: [],
 110: [],


In [75]:
#该模板不准确！
def template_no7():
    #解决“出了哪些”的问题
    template = "(.+)的全称(是什么)?.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group().lower())
            e1 = match_obj.group(1).lower()

            #查询库
            session = driver.session()
            cql = "match (a:Entity)-[b:Relation]-(c:Entity) \
                        where a.name=$name and b.name=~'.*[缩|简|称].*'  return c.name"

            result = session.run(cql,name='<'+e1+'>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

In [76]:
#该模板不准确！
def template_no8():
    #解决“出了哪些”的问题
    template = "([^的]+)(的)?股票代码.*"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group().lower())
            e1 = match_obj.group(1).lower()

            #查询库
            session = driver.session()
            cql = "match (a:Entity)-[b:Relation]-(c:Entity) \
                        where a.name=$name and b.name='<股票代码>'  return c.name"

            result = session.run(cql,name='<'+e1+'>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict

## 写了八个模板

In [79]:
def template_no9():
    #解决“出了哪些”的问题
    template = "(.*)家？"
    count=0
    result_dict={}

    for i in predata:
        q = predata[i]['question']
        result_dict[i]=[]
        match_obj = re.match(template, q)
        if match_obj:
            print(i,match_obj.group().lower())
            e1 = match_obj.group(1).lower()

            #查询库
            session = driver.session()
            cql = "match (a:Entity)-[b:Relation]-(c:Entity) \
                        where a.name=$name and b.name='<股票代码>'  return c.name"

            result = session.run(cql,name='<'+e1+'>')
            for x in result:
                print(x['c.name'])
                result_dict[i].append(x['c.name'])

            count+=1
    print('此类问题所占比例为：'+str(100*count/len(predata)), str(count)+'/'+str(len(predata)))
    return result_dict
template_no9()

216 北京大学出了哪些作家？
220 哈佛大学出了哪些物理学家？
229 "现代遗传学之父"出生在哪个国家？
248 清华大学出了哪些化学家？
252 重庆大学出过哪些企业家？
256 北京大学出了哪些天文学家？
266 历史上西南联大培养了哪些科学家？
287 我国在明朝时都有哪些著名的外交家？
291 浙江大学出了哪些科学家？
294 北京大学出了哪些科学家？
296 武汉大学出了哪些天文学家？
298 北京大学出了哪些经济学家？
305 吉林大学出过哪些科学家？
326 武汉大学出了哪些企业家？
329 清华大学出了哪些哲学家？
335 英国有哪些著名的数学家？
336 清华大学出了哪些天文学家？
345 北京大学出了哪些物理学家？
346 北京大学出了哪些思想家？
357 四川大学出了哪些作家？
372 四川大学出了哪些企业家？
383 北京大学出了哪些生物学家？
395 东南大学出过哪些数学家？
397 吉林大学出过哪些企业家？
429 恒生电子的兄弟企业有哪几家？
643 江苏省的上市公司一共有多少家？
698 与海通证券董事长关联的公司有几家？
712 在国内和香港都已上市的保险公司有哪几家？
724 《三个音乐家》的作者是什么派别的画家？
746 计算机的发明者，被称为计算机之父的是哪位科学家？
此类问题所占比例为：3.91644908616188 30/766


{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [],
 65: [],
 66: [],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [],
 79: [],
 80: [],
 81: [],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: [],
 88: [],
 89: [],
 90: [],
 91: [],
 92: [],
 93: [],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [],
 100: [],
 101: [],
 102: [],
 103: [],
 104: [],
 105: [],
 106: [],
 107: [],
 108: [],
 109: [],
 110: [],


In [77]:
def write2file(source,target,dic):
    source = open(source,'r',encoding='utf-8').readlines()
    saveto = open(target,'w',encoding='utf-8')
    for i in range(len(source)):
        if dic[i] !=[]:
            saveto.write('\t'.join(dic[i])+'\n')
        else:
            saveto.write(source[i])
    saveto.close()

In [78]:
write2file('result/3in1.txt','result/result1.txt',template_no1())
write2file('result/result1.txt','result/result2.txt',template_no2())
write2file('result/result2.txt','result/result3.txt',template_no3())
write2file('result/result3.txt','result/result4.txt',template_no4())
write2file('result/result4.txt','result/result5.txt',template_no5())
write2file('result/result5.txt','result/result6.txt',template_no6())
write2file('result/result6.txt','result/result7.txt',template_no7())
write2file('result/result7.txt','result/result8.txt',template_no8())

216 北京大学 作家
216 北京大学 作家_（汉语词语）
<余中先>
<杨松河>
<常聪>
<桐华>
<夏笳>
<李华亭_（中国当代作家）>
<骆漠>
<胡兰波>
<项小米>
<儒帅哲师>
<刘大军_（北京某文化发展有限公司副总经理）>
<顾随>
<杜染>
<倪厚玉>
<史杰鹏>
<王蓬_（陕西省作家协会副主席）>
<柳鸣九>
<后山>
<白崇义>
<黄培佳>
<李翊云>
<左泓>
<刘震云_（中国内地作家）>
<马自力>
<张志春_（易学家）>
<张承志_（中国著名作家，红卫兵创始人之一）>
<白榕_（中国内地作家）>
<石丛>
<查建英>
<戈悟觉>
<赵曙光_（中国内地作家）>
<黄蓓佳>
<冼宁_（中国现当代作家）>
<赵遐秋>
<废名_（20世纪著名作家、学者）>
<胡又天>
<陈楸帆>
<徐訏>
<沈威风>
<王实味>
<许君远>
<娇羞胡萝卜>
<王沂暖>
<马昌仪>
<马嘶_（原唐山市文联主席）>
<马宪泉>
<杜承南>
<董志谋>
<郑恩波>
<张天翼_（中国现代著名作家）>
<陈炜谟>
<金涛_（中国科普作家协会副理事长）>
<陆侃如>
<西川_（中国当代诗人）>
<悠世>
<张伯江>
<俞平伯>
<刘海影>
<刘咏_（中国当代作家）>
<韩耀成>
<树下野狐>
<贺捷生>
220 哈佛大学 物理学家
220 哈佛大学 物理学家_（探索、研究物理学的科学家）
<罗伯特·奥本海默>
<叶企孙>
<戴维·波利策>
<珀西·布里奇曼>
<西奥多·赖曼>
<胡刚复>
<菲利普·沃伦·安德森>
248 清华大学 化学家
248 清华大学 化学家
<王晓业>
<庄亚辉>
<徐又一>
<季鸣时>
<金道森>
252 重庆大学 企业家
252 重庆大学 企业家
<任正非>
<董德福_（云狐时代集团董事长）>
256 北京大学 天文学家
256 北京大学 天文学家
<陈建生_（中国信息经济学会理事）>
291 浙江大学 科学家
291 浙江大学 科学家_（从事科学研究的人群）
<吴剑锋_（中国科学院理化技术研究所副所长）>
<林励吾>
<谭其猛>
<吕鸿声>
<罗安_（中国工程院院士）>
<李志坚_（中国科学院院士）>
<李光林_（中国科学院深圳先进技术研究院研究员）>
<文学军>
294 北京大学 科学家
294 北京大学 科学家_（从事科学研究的人群）
<

In [41]:
men2ent_dic['威尔药业']

[['威尔药业', '0']]